# Slowly Changing Dimensions made Easy with Durable Keys - Demos

This series of demo's walks through how we can add a durable key to an existing dimension, how we adjust our merge statements to keep it up to date, and how we add that durable key into the fact table to allow easy As-posted, As-is and As-was queries.

- As-is – Shows data using current structure
- As-posted – Shows data using the “historical truth”, eg. the structure at the time that data point happened
- As-of – Shows data using structure at point in time, eg. 1st Jan 2020


***Note***: *None of the tables are indexed as they are so small, index as appropriate in your systems!*


## 1) Setup the demo environment by running the setup script 'Slowly Changing Dimensions made Easy with Durable Keys/Demos/1 - Setup Demo Environment.sql'
The setup script uses dynamic data generation, and therefore aggregation results will be different each time you rebuild the demo database.

## 2) Add a durable key to the product dimension

In [20]:
-- If the Db got rebuilt, the connection is reset, and this will error without breaking the rest of the notebook!
USE DurableKeyDemo

In [21]:
USE DurableKeyDemo
GO

CREATE SEQUENCE dim.CustomerDK AS INT
    START WITH 1 INCREMENT BY 1
GO

ALTER TABLE dim.Customer ADD CustomerDK INT NULL DEFAULT (NEXT VALUE FOR dim.CustomerDK);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.059

## 3) Populate the durable key for existing values

We group the business keys and assign a durable key to each group using the sequence. We put this in a temp table (becuase there are limitations on where you can use the `NEXT VALUE FOR` syntax, we can't just use a CTE or sub-query), and use the temp table to update the dimension.

***Note:*** *If your business keys have changed over time (eg a new source system added, etc), this may be more complex than the join on business keys below.*

In [22]:
SELECT *
FROM dim.Customer;

DROP TABLE IF EXISTS #customerDK;

SELECT c.SourceSystem,
       c.SourceId,
       NEXT VALUE FOR dim.CustomerDK AS CustomerDK
INTO #customerDK
FROM dim.Customer AS c
GROUP BY SourceSystem,
         SourceId;


UPDATE c
SET CustomerDK = dk.CustomerDK
FROM dim.Customer AS c
INNER JOIN #customerDK AS dk
  ON dk.SourceSystem = c.SourceSystem
     AND dk.SourceId = c.SourceId;

SELECT *
FROM dim.Customer;


(7 rows affected)

(4 rows affected)

(7 rows affected)

(7 rows affected)

Total execution time: 00:00:00.088

CustomerSK,SourceSystem,SourceId,CustomerName,CustomerEmail,CustomerCategory,ValidFromDate,ValidToDate,IsCurrentRow,MergeCheckSum,CustomerDK
1,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Regular,2018-02-18,2018-11-11,0,1467799504,NULL
2,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Big Fish,2018-11-12,9999-12-31,1,-2119939865,NULL
3,Website,1,Niall Langley,niall.langley@gmail.com,New Customer,2019-04-19,2019-07-03,0,878283563,NULL
4,Website,2,Alfred,alfred@batman.com,Flavour Dabbler,2019-04-19,9999-12-31,1,-219148910,NULL
5,Website,3,Arthur Dent,arthur.dent@somewhereoutthere.com,Irregular Customer,2019-04-19,9999-12-31,1,-1177136545,NULL
6,Website,1,Niall Langley,niall.langley@gmail.com,Regular Customer,2019-07-04,2020-04-30,0,1606136978,NULL
7,Website,1,Niall Langley,niall.langley@gmail.com,Super Customer,2020-05-01,9999-12-31,1,1056696475,NULL


CustomerSK,SourceSystem,SourceId,CustomerName,CustomerEmail,CustomerCategory,ValidFromDate,ValidToDate,IsCurrentRow,MergeCheckSum,CustomerDK
1,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Regular,2018-02-18,2018-11-11,0,1467799504,1
2,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Big Fish,2018-11-12,9999-12-31,1,-2119939865,1
3,Website,1,Niall Langley,niall.langley@gmail.com,New Customer,2019-04-19,2019-07-03,0,878283563,2
4,Website,2,Alfred,alfred@batman.com,Flavour Dabbler,2019-04-19,9999-12-31,1,-219148910,3
5,Website,3,Arthur Dent,arthur.dent@somewhereoutthere.com,Irregular Customer,2019-04-19,9999-12-31,1,-1177136545,4
6,Website,1,Niall Langley,niall.langley@gmail.com,Regular Customer,2019-07-04,2020-04-30,0,1606136978,2
7,Website,1,Niall Langley,niall.langley@gmail.com,Super Customer,2020-05-01,9999-12-31,1,1056696475,2


## 4) Update the durable key column to make it not nullable

In [23]:
ALTER TABLE dim.Customer ALTER COLUMN CustomerDK INT NOT NULL;
GO

CREATE UNIQUE INDEX UQ_dim_Customer_#CustomerDK ON dim.Customer(CustomerDK)
	INCLUDE	(CustomerName, CustomerEmail, CustomerCategory)
	WHERE (IsCurrentRow = 1)
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.019

## 5) Change our merge statement to add a new customer, and update and existing one

- The new customer should get assigned a new durable key from the default
- The existing customer should keep the durable key assigned to the previous record

Note: For more info on the insert/merge systax used here see [this blog](https://www.mssqltips.com/sqlservertip/2883/using-the-sql-server-merge-statement-to-process-type-2-slowly-changing-dimensions/).

In [24]:
SELECT *
FROM dim.Customer;

DROP TABLE IF EXISTS #customerUpdates;

SELECT *,
       CheckSum(CustomerName, CustomerEmail, CustomerCategory) AS MergeCheckSum
INTO #customerUpdates
FROM (VALUES 
        ('Website', 1, 'Niall Langley',     'niall.langley@gmail.com',              'Advocate'),
        ('Website', 2, 'Alfred',            'alfred@batman.com',                    'Flavour Dabbler'),
        ('Website', 3, 'Arthur Dent',       'arthur.dent@somewhereoutthere.com',    'Irregular Orders'),
        ('Website', 4, 'The Queen',         'lizzy@gov.uk',                         'VIP')
     )AS v (SourceSystem, SourceId, CustomerName, CustomerEmail, CustomerCategory)



-- Use a transaction as the merge statement does seperate statements under the covers
BEGIN TRANSACTION;

INSERT INTO dim.Customer (
    SourceSystem,
    SourceId,
    CustomerName,
    CustomerEmail,
    CustomerCategory,
    CustomerDK                                  -- ADDED THIS LINE FOR DURABLE KEY SUPPORT
)
SELECT SourceSystem,
       SourceId,
       CustomerName,
       CustomerEmail,
       CustomerCategory,
       CustomerDK                               -- ADDED THIS LINE FOR DURABLE KEY SUPPORT
FROM (
    MERGE dim.Customer AS t
    USING (SELECT *
           FROM #customerUpdates
          )AS s
    ON t.SourceSystem = s.SourceSystem
       AND t.SourceId = s.SourceId
    WHEN NOT MATCHED
      THEN INSERT (SourceSystem, SourceId, CustomerName, CustomerEmail, CustomerCategory)
           VALUES (SourceSystem, SourceId, CustomerName, CustomerEmail, CustomerCategory)
    WHEN MATCHED
         AND t.MergeCheckSum <> s.MergeCheckSum
         AND t.IsCurrentRow = 1
      THEN UPDATE
           SET ValidToDate = DATEADD(DAY, -1, CAST(GETDATE() AS DATE)),
               IsCurrentRow = 0
    OUTPUT $action,
           s.SourceSystem,
           s.SourceId,
           s.CustomerName,
           s.CustomerEmail,
           s.CustomerCategory,
           INSERTED.CustomerDK                  -- ADDED THIS LINE FOR DURABLE KEY SUPPORT
) AS changes (
    MergeAction,
    SourceSystem,
    SourceId,
    CustomerName,
    CustomerEmail,
    CustomerCategory,
    CustomerDK                                  -- ADDED THIS LINE FOR DURABLE KEY SUPPORT
)
WHERE MergeAction = 'UPDATE';

COMMIT


SELECT *
FROM dim.Customer;

(7 rows affected)

(4 rows affected)

(2 rows affected)

(10 rows affected)

Total execution time: 00:00:00.163

CustomerSK,SourceSystem,SourceId,CustomerName,CustomerEmail,CustomerCategory,ValidFromDate,ValidToDate,IsCurrentRow,MergeCheckSum,CustomerDK
1,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Regular,2018-02-18,2018-11-11,0,1467799504,1
2,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Big Fish,2018-11-12,9999-12-31,1,-2119939865,1
3,Website,1,Niall Langley,niall.langley@gmail.com,New Customer,2019-04-19,2019-07-03,0,878283563,2
4,Website,2,Alfred,alfred@batman.com,Flavour Dabbler,2019-04-19,9999-12-31,1,-219148910,3
5,Website,3,Arthur Dent,arthur.dent@somewhereoutthere.com,Irregular Customer,2019-04-19,9999-12-31,1,-1177136545,4
6,Website,1,Niall Langley,niall.langley@gmail.com,Regular Customer,2019-07-04,2020-04-30,0,1606136978,2
7,Website,1,Niall Langley,niall.langley@gmail.com,Super Customer,2020-05-01,9999-12-31,1,1056696475,2


CustomerSK,SourceSystem,SourceId,CustomerName,CustomerEmail,CustomerCategory,ValidFromDate,ValidToDate,IsCurrentRow,MergeCheckSum,CustomerDK
1,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Regular,2018-02-18,2018-11-11,0,1467799504,1
2,Trade,1,The tea shop,acounts@theteashop.co.uk,Trade - Big Fish,2018-11-12,9999-12-31,1,-2119939865,1
3,Website,1,Niall Langley,niall.langley@gmail.com,New Customer,2019-04-19,2019-07-03,0,878283563,2
4,Website,2,Alfred,alfred@batman.com,Flavour Dabbler,2019-04-19,9999-12-31,1,-219148910,3
5,Website,3,Arthur Dent,arthur.dent@somewhereoutthere.com,Irregular Customer,2019-04-19,2020-09-06,0,-1177136545,4
6,Website,1,Niall Langley,niall.langley@gmail.com,Regular Customer,2019-07-04,2020-04-30,0,1606136978,2
7,Website,1,Niall Langley,niall.langley@gmail.com,Super Customer,2020-05-01,2020-09-06,0,1056696475,2
8,Website,4,The Queen,lizzy@gov.uk,VIP,2020-09-07,9999-12-31,1,1766473576,5
9,Website,3,Arthur Dent,arthur.dent@somewhereoutthere.com,Irregular Orders,2020-09-07,9999-12-31,1,1437106049,4
10,Website,1,Niall Langley,niall.langley@gmail.com,Advocate,2020-09-07,9999-12-31,1,-413305737,2


## 6) Add a view for the current version of the customer, giving a view like a SCD Type 1

In [25]:
DROP VIEW IF EXISTS dim.CustomerCurrent;
GO

CREATE VIEW dim.CustomerCurrent
AS
SELECT CustomerDK,
       CustomerName AS CurrentCustomerName,
       CustomerEmail AS CurrentCustomerEmail,
       CustomerCategory AS CurrentCustomerCategory
FROM dim.Customer
WHERE IsCurrentRow = 1;
GO

SELECT *
FROM dim.CustomerCurrent;

Commands completed successfully.

Commands completed successfully.

(5 rows affected)

Total execution time: 00:00:00.041

CustomerDK,CurrentCustomerName,CurrentCustomerEmail,CurrentCustomerCategory
1,The tea shop,acounts@theteashop.co.uk,Trade - Big Fish
3,Alfred,alfred@batman.com,Flavour Dabbler
5,The Queen,lizzy@gov.uk,VIP
4,Arthur Dent,arthur.dent@somewhereoutthere.com,Irregular Orders
2,Niall Langley,niall.langley@gmail.com,Advocate


## 7) Add the durable key to the fact table, populating it using the existing surrogate key link

In [26]:
SELECT TOP 100 *
FROM fact.Sales
WHERE CustomerSK > 2
      AND OrderDateSK > 20200000

ALTER TABLE fact.Sales ADD CustomerDK INT NULL;
GO

UPDATE s
SET CustomerDK = c.CustomerDK
FROM fact.Sales AS s
INNER JOIN dim.Customer AS c
  ON c.CustomerSK = s.CustomerSK;
GO

ALTER TABLE fact.Sales ALTER COLUMN CustomerDK INT NOT NULL;
GO

SELECT TOP 100 *
FROM fact.Sales
WHERE CustomerSK > 2
      AND OrderDateSK > 20200000


(6 rows affected)

(192 rows affected)

CustomerSK,ProductSK,OrderDateSK,ShipDateSK,OrderId,OrderLineId,Quantity,PricePaidPerUnit
5,3,20200502,20200502,187,1,1,3.9900
5,13,20200829,20200831,214,1,1,3.5900
6,1,20200215,20200217,168,1,1,10.3900
6,1,20200331,20200402,178,1,1,11.6900
7,1,20200516,20200518,191,1,1,11.6900
7,10,20200701,20200702,199,1,1,12.9900


Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.221

CustomerSK,ProductSK,OrderDateSK,ShipDateSK,OrderId,OrderLineId,Quantity,PricePaidPerUnit,CustomerDK
5,3,20200502,20200502,187,1,1,3.9900,4
5,13,20200829,20200831,214,1,1,3.5900,4
6,1,20200215,20200217,168,1,1,10.3900,2
6,1,20200331,20200402,178,1,1,11.6900,2
7,1,20200516,20200518,191,1,1,11.6900,2
7,10,20200701,20200702,199,1,1,12.9900,2
